# Stationary Two-Group Pedestrian Flow

## Strong Form Formulation

For two pedestrian groups (i = 1, 2):

**Continuity Equation (for each group):**
$$
\nabla \cdot (-\varepsilon\nabla\rho_i + \rho_i u_i) = 0 \quad \text{in } \Omega
$$

**Helmholtz Equation (for each group):**
$$
\Delta\psi_i - \frac{1}{\delta^2 f_i^2(\rho)} \psi_i = 0 \quad \text{in } \Omega
$$

**Velocity Field (for each group):**
$$
u_i = f_i(\rho) \frac{\nabla\psi_i}{\|\nabla\psi_i\|}
$$

**Total Density (coupling):**
$$
\rho = \rho_1 + \rho_2
$$

**Key point:** Both groups feel the total density ρ in their speed f_i(ρ) and Helmholtz equation!

### Boundary Conditions (for each group i)

**On Walls:**
- Density: $(-\varepsilon\nabla\rho_i + \rho_i u_i)\cdot n = 0$
- Potential: $\nabla\psi_i\cdot n = 0$

**On Exits:**
- Density: $(-\varepsilon\nabla\rho_i)\cdot n = 0$
- Potential: $\psi_i = 1$

**On Entrances:**
- Density: $-(-\varepsilon\nabla\rho_i + \rho_i u_i)\cdot n = g_i$
- Potential: $(u_0\delta\nabla\psi_i)\cdot n + \psi_i = 0$

---

## Weak Form Formulation

### Helmholtz Equation (for each group i)

Find $\psi_i \in V_\psi$ such that for all $\varphi \in V_{0,\psi}$:
$$
\boxed{
\int_\Omega \nabla\psi_i \cdot \nabla\varphi \, d\Omega + \int_\Omega \kappa_i^2(\rho) \psi_i\varphi \, d\Omega + \int_{\Gamma_{\text{entry}}} \frac{1}{u_0\delta} \psi_i\varphi \, dS = 0
}
$$

where $\kappa_i(\rho) = \frac{1}{\delta f_i(\rho)}$ and $\rho = \rho_1 + \rho_2$

**Essential BC:** $\psi_i = 1$ on $\Gamma_{\text{exit}}$

### Continuity Equation (for each group i)

Find $\rho_i \in V_\rho$ such that for all $w \in V_\rho$:
$$
\boxed{
\int_\Omega \varepsilon\nabla\rho_i \cdot \nabla w \, d\Omega - \int_\Omega (\rho_i u_i) \cdot \nabla w \, d\Omega + \int_{\Gamma_{\text{exit}}} (\rho_i u_i \cdot n) w \, dS = \int_{\Gamma_{\text{entry}}} g_i w \, dS
}
$$

**No essential BCs**

---

## Picard Iteration Algorithm

For iteration k = 0, 1, 2, ...:

1. **Compute total density:** $\rho^{(k)} = \rho_1^{(k)} + \rho_2^{(k)}$

2. **For each group i = 1, 2:**
   - Solve Helmholtz for $\psi_i^{(k+1)}$ using $\rho^{(k)}$
   - Compute velocity $u_i^{(k+1)} = f_i(\rho^{(k)}) \frac{\nabla\psi_i^{(k+1)}}{\|\nabla\psi_i^{(k+1)}\| + \eta}$
   - Solve Continuity for $\rho_i^{(k+1)}$ using $u_i^{(k+1)}$

3. **Check convergence:** $\max(\|\rho_1^{(k+1)} - \rho_1^{(k)}\|, \|\rho_2^{(k+1)} - \rho_2^{(k)}\|) < \text{tol}$

In [ ]:
# ========================================
# Module Import
# ========================================

from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import numpy as np

In [ ]:
# ========================================
# Geometry and BC naming
# ========================================

mesh_maxh = 0.05
Hcol = 1.0  # Height [m]
Hwid = 1.0  # Width [m]

# Top rectangle
rect = Rectangle(Hwid, Hcol).Face()
rect.edges.Min(X).name = "entry_group1"   # left edge
rect.edges.Max(X).name = "entry_group2"    # right edge
rect.edges.Max(Y).name = "exit_group1"    # top edge
rect.edges.Min(Y).name = "exit_group2"      # bottom edge

geom = OCCGeometry(rect, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=mesh_maxh))

print(f"Mesh: {mesh.nv} vertices, {mesh.ne} elements")

In [ ]:
# ========================================
# Physical Parameters (from Weidmann)
# ========================================
u0 = 1.36          # Free-flow walking speed [m/s]
rho_c = 8.0        # Critical density [ped/m²]
gamma_w = 1.913    # Weidmann shape parameter [ped/m²]

# ========================================
# Regularization Parameters
# ========================================
delta = 0.1        # Viscosity parameter [m]
epsilon = 0.1      # Diffusion coefficient [m²]
eta = 1e-8         # Gradient regularization (avoid division by zero)

# ========================================
# Boundary Conditions (for each group)
# ========================================
g_inflow_1 = 0.6   # Group 1 influx at entrance [ped/(m·s)]
g_inflow_2 = 0.4   # Group 2 influx at entrance [ped/(m·s)]

# ========================================
# Numerical Parameters
# ========================================
p_order = 3        # Polynomial order of finite element space
max_iter = 100     # Maximum number of Picard iterations
tol = 1e-6         # Convergence tolerance

print("Model Parameters:")
print(f"  u0 (free-flow speed)     = {u0} m/s")
print(f"  rho_c (critical density) = {rho_c} ped/m²")
print(f"  gamma (Weidmann param)   = {gamma_w} ped/m²")
print(f"\nRegularization:")
print(f"  delta (viscosity)        = {delta} m")
print(f"  epsilon (diffusion)      = {epsilon} m²")
print(f"  eta (grad regularization)= {eta}")
print(f"\nBoundary Conditions:")
print(f"  g_inflow_1 (Group 1)     = {g_inflow_1} ped/(m·s)")
print(f"  g_inflow_2 (Group 2)     = {g_inflow_2} ped/(m·s)")
print(f"  Total inflow             = {g_inflow_1 + g_inflow_2} ped/(m·s)")
print(f"\nNumerical Settings:")
print(f"  max_iter                 = {max_iter}")
print(f"  tolerance                = {tol}")

In [ ]:
# ============================================================================
# PARAMETER ANALYSIS - Verify numerical stability before running solver
# ============================================================================
from src import analyze_parameters

print("Running parameter analysis for two-group simulation...\n")
print("Note: Analyzing with total inflow (both groups combined)\n")

results = analyze_parameters(
    u0=u0,
    rho_c=rho_c,
    gamma_w=gamma_w,
    delta=delta,
    epsilon=epsilon,
    mesh=mesh,
    mesh_maxh=mesh_maxh,
    p_order=p_order,
    Hwid=Hwid,
    Hcol=Hcol,
    omega=None  # No underrelaxation in standard Picard
)

# Check if parameters are acceptable
if results['stability_continuity'] in ['EXCELLENT', 'GOOD']:
    print("\n" + "="*60)
    print("✓✓ Parameters are GOOD! Ready to proceed with simulation.")
    print("="*60)
else:
    print("\n" + "="*60)
    print("⚠⚠ WARNING: Parameters may cause stability issues!")
    print("    Consider adjusting epsilon, h, or p before continuing.")
    print("="*60)

In [ ]:
# ========================================
# Weidmann Fundamental Diagram
# ========================================

def weidmann_speed(rho_val):
    """
    Weidmann fundamental diagram: f(rho)
    
    Returns walking speed as a function of density.
    Both groups use the same fundamental diagram.
    
    f(rho) = u0 * (1 - exp(-gamma * (1/rho - 1/rho_c)))  for 0 < rho <= rho_c
    f(0)   = u0                                           for rho = 0
    
    Parameters:
    -----------
    rho_val : float or CoefficientFunction
        Total pedestrian density [ped/m²]
    
    Returns:
    --------
    speed : float or CoefficientFunction
        Walking speed [m/s]
    """
    # Add small regularization to avoid division by zero
    rho_reg = IfPos(rho_val - 1e-10, rho_val, 1e-10)
    
    # Weidmann formula
    speed = u0 * (1 - exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    
    # Ensure speed doesn't exceed u0 (free-flow speed)
    speed = IfPos(u0 - speed, speed, u0)
    
    # Ensure speed is non-negative
    speed = IfPos(speed, speed, 0.0)
    
    return speed


# Test function for pure Python evaluation
def weidmann_speed_python(rho_val):
    """Python version for numerical testing"""
    rho_reg = max(rho_val, 1e-10)
    speed = u0 * (1 - np.exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    speed = min(max(speed, 0.0), u0)
    return speed


# Test the function
print("Testing Weidmann fundamental diagram:")
print(f"  f(0.1) = {weidmann_speed_python(0.1):.4f} m/s")
print(f"  f(1.0) = {weidmann_speed_python(1.0):.4f} m/s")
print(f"  f(2.0) = {weidmann_speed_python(2.0):.4f} m/s")
print(f"  f(4.0) = {weidmann_speed_python(4.0):.4f} m/s")
print(f"  f(8.0) = {weidmann_speed_python(rho_c):.4f} m/s  (at critical density)")

In [ ]:
# ========================================
# Finite Element Spaces
# ========================================

# H1 spaces for density (rho_1, rho_2) - no Dirichlet BCs
fes_rho1 = H1(mesh, order=p_order)
fes_rho2 = H1(mesh, order=p_order)

# H1 spaces for potential (psi_1, psi_2) - Dirichlet BC on respective exits
fes_psi1 = H1(mesh, order=p_order, dirichlet="exit_group1")
fes_psi2 = H1(mesh, order=p_order, dirichlet="exit_group2")

# Vector H1 spaces for velocity fields (u_1, u_2)
fes_u1 = H1(mesh, order=p_order, dim=2)
fes_u2 = H1(mesh, order=p_order, dim=2)

# H1 space for total density
fes_rho_total = H1(mesh, order=p_order)

print("Finite Element Spaces:")
print(f"  Group 1:")
print(f"    Density (ρ₁):   {fes_rho1.ndof} DOFs")
print(f"    Potential (ψ₁): {fes_psi1.ndof} DOFs")
print(f"    Velocity (u₁):  {fes_u1.ndof} DOFs")
print(f"  Group 2:")
print(f"    Density (ρ₂):   {fes_rho2.ndof} DOFs")
print(f"    Potential (ψ₂): {fes_psi2.ndof} DOFs")
print(f"    Velocity (u₂):  {fes_u2.ndof} DOFs")
print(f"  Total density (ρ): {fes_rho_total.ndof} DOFs")

# ========================================
# Grid Functions (Solution Variables)
# ========================================

# Group 1
gf_rho1 = GridFunction(fes_rho1, name="density_1")
gf_psi1 = GridFunction(fes_psi1, name="potential_1")
gf_u1 = GridFunction(fes_u1, name="velocity_1")
gf_rho1_old = GridFunction(fes_rho1, name="density_1_old")

# Group 2
gf_rho2 = GridFunction(fes_rho2, name="density_2")
gf_psi2 = GridFunction(fes_psi2, name="potential_2")
gf_u2 = GridFunction(fes_u2, name="velocity_2")
gf_rho2_old = GridFunction(fes_rho2, name="density_2_old")

# Total density (coupling)
gf_rho_total = GridFunction(fes_rho_total, name="density_total")

# ========================================
# Initialize with reasonable values
# ========================================

# Initialize densities with small positive values
gf_rho1.Set(0.05)
gf_rho1_old.Set(0.05)

gf_rho2.Set(0.05)
gf_rho2_old.Set(0.05)

# Initialize total density
gf_rho_total.Set(0.1)

# Initialize potentials with linear interpolation
# Group 1: enters from left (x=0), exits at top (y=Hcol)
gf_psi1.Set(y / Hcol)
gf_psi1.Set(1.0, definedon=mesh.Boundaries("exit_group1"))

# Group 2: enters from right (x=Hwid), exits at bottom (y=0)
gf_psi2.Set((Hcol - y) / Hcol)
gf_psi2.Set(1.0, definedon=mesh.Boundaries("exit_group2"))

# Velocities will be computed from psi
gf_u1.Set((0, 0))
gf_u2.Set((0, 0))

print("\nGrid Functions initialized:")
print(f"  ρ₁: min = {min(gf_rho1.vec):.4f}, max = {max(gf_rho1.vec):.4f}")
print(f"  ρ₂: min = {min(gf_rho2.vec):.4f}, max = {max(gf_rho2.vec):.4f}")
print(f"  ψ₁: min = {min(gf_psi1.vec):.4f}, max = {max(gf_psi1.vec):.4f}")
print(f"  ψ₂: min = {min(gf_psi2.vec):.4f}, max = {max(gf_psi2.vec):.4f}")

In [ ]:
# ========================================
# Velocity Field Computation
# ========================================

def compute_velocity_group1():
    """
    Compute velocity field for Group 1.
    u₁ = f(ρ_total) * ∇ψ₁ / (||∇ψ₁|| + η)
    """
    grad_psi = grad(gf_psi1)
    grad_psi_norm = sqrt(grad_psi[0]**2 + grad_psi[1]**2 + eta)
    speed = weidmann_speed(gf_rho_total)  # Uses TOTAL density!
    velocity_cf = speed * grad_psi / grad_psi_norm
    gf_u1.Set(velocity_cf)
    return gf_u1


def compute_velocity_group2():
    """
    Compute velocity field for Group 2.
    u₂ = f(ρ_total) * ∇ψ₂ / (||∇ψ₂|| + η)
    """
    grad_psi = grad(gf_psi2)
    grad_psi_norm = sqrt(grad_psi[0]**2 + grad_psi[1]**2 + eta)
    speed = weidmann_speed(gf_rho_total)  # Uses TOTAL density!
    velocity_cf = speed * grad_psi / grad_psi_norm
    gf_u2.Set(velocity_cf)
    return gf_u2


print("Velocity field computation functions defined:")
print("  u₁ = f(ρ_total) * ∇ψ₁ / (||∇ψ₁|| + η)")
print("  u₂ = f(ρ_total) * ∇ψ₂ / (||∇ψ₂|| + η)")
print("\n✓ Velocity computation ready")

In [ ]:
# ========================================
# Picard Iteration Solver (Two Groups)
# ========================================

def solve_twogroup_picard():
    """
    Solve the coupled two-group pedestrian flow problem using Picard iteration.
    
    Algorithm:
    1. Compute total density: ρ = ρ₁ + ρ₂
    2. For each group i=1,2:
       a. Solve Helmholtz for ψᵢ (using ρ_total)
       b. Compute velocity uᵢ from ψᵢ (using ρ_total)
       c. Solve Continuity for ρᵢ (using uᵢ)
    3. Check convergence
    """
    
    print("=" * 60)
    print("Starting Two-Group Picard Iteration")
    print("=" * 60)
    
    residuals = []
    
    for iteration in range(max_iter):
        
        # Store old densities for convergence check
        gf_rho1_old.vec.data = gf_rho1.vec
        gf_rho2_old.vec.data = gf_rho2.vec
        
        # ====================================
        # Step 1: Update total density
        # ====================================
        gf_rho_total.Set(gf_rho1 + gf_rho2)
        
        # ====================================
        # GROUP 1
        # ====================================
        
        # Solve Helmholtz for psi1
        f_rho = weidmann_speed(gf_rho_total)
        kappa_sq = 1.0 / (delta**2 * f_rho**2)
        
        psi1 = fes_psi1.TrialFunction()
        phi1 = fes_psi1.TestFunction()
        
        a_psi1 = BilinearForm(fes_psi1, symmetric=True)
        a_psi1 += grad(psi1) * grad(phi1) * dx
        a_psi1 += kappa_sq * psi1 * phi1 * dx
        a_psi1 += (1.0 / (u0 * delta)) * psi1 * phi1 * ds("entry_group1")
        
        L_psi1 = LinearForm(fes_psi1)
        
        a_psi1.Assemble()
        L_psi1.Assemble()
        
        gf_psi1.Set(1.0, definedon=mesh.Boundaries("exit_group1"))
        gf_psi1.vec.data += a_psi1.mat.Inverse(fes_psi1.FreeDofs()) * (L_psi1.vec - a_psi1.mat * gf_psi1.vec)
        
        # Compute velocity for group 1
        compute_velocity_group1()
        
        # Solve Continuity for rho1
        rho1 = fes_rho1.TrialFunction()
        w1 = fes_rho1.TestFunction()
        
        a_rho1 = BilinearForm(fes_rho1, symmetric=False)
        a_rho1 += epsilon * grad(rho1) * grad(w1) * dx
        a_rho1 += -rho1 * (gf_u1 * grad(w1)) * dx
        n = specialcf.normal(2)
        a_rho1 += rho1 * (gf_u1 * n) * w1 * ds("exit_group1")
        
        L_rho1 = LinearForm(fes_rho1)
        L_rho1 += g_inflow_1 * w1 * ds("entry_group1")
        
        with TaskManager():
            a_rho1.Assemble()
            L_rho1.Assemble()
            gf_rho1.vec.data = a_rho1.mat.Inverse(fes_rho1.FreeDofs()) * L_rho1.vec
        
        # ====================================
        # GROUP 2
        # ====================================
        
        # Solve Helmholtz for psi2 (using same kappa_sq from total density)
        psi2 = fes_psi2.TrialFunction()
        phi2 = fes_psi2.TestFunction()
        
        a_psi2 = BilinearForm(fes_psi2, symmetric=True)
        a_psi2 += grad(psi2) * grad(phi2) * dx
        a_psi2 += kappa_sq * psi2 * phi2 * dx
        a_psi2 += (1.0 / (u0 * delta)) * psi2 * phi2 * ds("entry_group2")
        
        L_psi2 = LinearForm(fes_psi2)
        
        a_psi2.Assemble()
        L_psi2.Assemble()
        
        gf_psi2.Set(1.0, definedon=mesh.Boundaries("exit_group2"))
        gf_psi2.vec.data += a_psi2.mat.Inverse(fes_psi2.FreeDofs()) * (L_psi2.vec - a_psi2.mat * gf_psi2.vec)
        
        # Compute velocity for group 2
        compute_velocity_group2()
        
        # Solve Continuity for rho2
        rho2 = fes_rho2.TrialFunction()
        w2 = fes_rho2.TestFunction()
        
        a_rho2 = BilinearForm(fes_rho2, symmetric=False)
        a_rho2 += epsilon * grad(rho2) * grad(w2) * dx
        a_rho2 += -rho2 * (gf_u2 * grad(w2)) * dx
        a_rho2 += rho2 * (gf_u2 * n) * w2 * ds("exit_group2")
        
        L_rho2 = LinearForm(fes_rho2)
        L_rho2 += g_inflow_2 * w2 * ds("entry_group2")
        
        with TaskManager():
            a_rho2.Assemble()
            L_rho2.Assemble()
            gf_rho2.vec.data = a_rho2.mat.Inverse(fes_rho2.FreeDofs()) * L_rho2.vec
        
        # ====================================
        # Check convergence
        # ====================================
        
        residual1_vec = gf_rho1.vec - gf_rho1_old.vec
        residual1 = sqrt(InnerProduct(residual1_vec, residual1_vec))
        
        residual2_vec = gf_rho2.vec - gf_rho2_old.vec
        residual2 = sqrt(InnerProduct(residual2_vec, residual2_vec))
        
        residual = max(residual1, residual2)
        residuals.append(residual)
        
        # Print progress
        if iteration % 1 == 0 or residual < tol:
            rho1_min, rho1_max = min(gf_rho1.vec), max(gf_rho1.vec)
            rho2_min, rho2_max = min(gf_rho2.vec), max(gf_rho2.vec)
            print(f"Iter {iteration:3d}: residual = {residual:.6e}")
            print(f"         ρ₁ ∈ [{rho1_min:.4f}, {rho1_max:.4f}], "
                  f"ρ₂ ∈ [{rho2_min:.4f}, {rho2_max:.4f}]")
        
        # Check convergence
        if residual < tol:
            print("=" * 60)
            print(f"✓ Converged in {iteration + 1} iterations!")
            print(f"  Final residual: {residual:.6e}")
            print("=" * 60)
            return True, iteration + 1, residual, residuals
    
    # Max iterations reached
    print("=" * 60)
    print(f"✗ Did not converge in {max_iter} iterations")
    print(f"  Final residual: {residual:.6e}")
    print("=" * 60)
    return False, max_iter, residual, residuals


# Run the solver
converged, n_iter, final_residual, residuals = solve_twogroup_picard()

In [ ]:
# ========================================
# Visualize Results
# ========================================

print("=" * 60)
print("Two-Group Solution Visualization")
print("=" * 60)

# Update total density
gf_rho_total.Set(gf_rho1 + gf_rho2)

# Compute domain area
domain_area = Integrate(1.0, mesh)

# Create order-1 space for min/max evaluation
# (Order-1 DOFs correspond to vertex values, making min/max meaningful)
fes_p1 = H1(mesh, order=1)

# Print solution statistics
print("\nSolution Statistics:")

print(f"\n  Group 1:")
rho1_integral = Integrate(gf_rho1 * dx, mesh)
rho1_mean = rho1_integral / domain_area
# Interpolate to P1 for min/max
gf_rho1_p1 = GridFunction(fes_p1)
gf_rho1_p1.Set(gf_rho1)
print(f"    Density (ρ₁): integral = {rho1_integral:.6f} ped")
print(f"                  mean = {rho1_mean:.6f} ped/m²")
print(f"                  min ≈ {min(gf_rho1_p1.vec):.6f} ped/m² (at vertices)")
print(f"                  max ≈ {max(gf_rho1_p1.vec):.6f} ped/m² (at vertices)")

print(f"\n  Group 2:")
rho2_integral = Integrate(gf_rho2 * dx, mesh)
rho2_mean = rho2_integral / domain_area
# Interpolate to P1 for min/max
gf_rho2_p1 = GridFunction(fes_p1)
gf_rho2_p1.Set(gf_rho2)
print(f"    Density (ρ₂): integral = {rho2_integral:.6f} ped")
print(f"                  mean = {rho2_mean:.6f} ped/m²")
print(f"                  min ≈ {min(gf_rho2_p1.vec):.6f} ped/m² (at vertices)")
print(f"                  max ≈ {max(gf_rho2_p1.vec):.6f} ped/m² (at vertices)")

print(f"\n  Total:")
rho_total_integral = Integrate(gf_rho_total * dx, mesh)
rho_total_mean = rho_total_integral / domain_area
# Interpolate to P1 for min/max
gf_rho_total_p1 = GridFunction(fes_p1)
gf_rho_total_p1.Set(gf_rho_total)
print(f"    Density (ρ):  integral = {rho_total_integral:.6f} ped")
print(f"                  mean = {rho_total_mean:.6f} ped/m²")
print(f"                  min ≈ {min(gf_rho_total_p1.vec):.6f} ped/m² (at vertices)")
print(f"                  max ≈ {max(gf_rho_total_p1.vec):.6f} ped/m² (at vertices)")

print(f"\n  Domain area:  {domain_area:.6f} m²")

print("=" * 60)

# Generate visualizations
print("\nGenerating visualizations...")

# Density fields
Draw(gf_rho1, mesh, "density_group_1")
print("  ✓ Density field - Group 1 (ρ₁)")

Draw(gf_rho2, mesh, "density_group_2")
print("  ✓ Density field - Group 2 (ρ₂)")

Draw(gf_rho_total, mesh, "density_total")
print("  ✓ Total density field (ρ)")

# Velocity vector fields
Draw(gf_u1, mesh, "velocity_group_1", vectors={"grid_size": 30})
print("  ✓ Velocity vector field - Group 1 (u₁)")

Draw(gf_u2, mesh, "velocity_group_2", vectors={"grid_size": 30})
print("  ✓ Velocity vector field - Group 2 (u₂)")

print("\n✓ Visualization complete!")